# Additionally evaluate models after training

## Warning: Does not use all values from wandb but data from current project state, make sure they are in sync, otherwise use testing_models.ipynb

In [1]:
# important for gpuhub
# !pip install -r ../../requirements.txt --upgrade

In [2]:
import wandb
import sys
import os

import torch
from torchvision import transforms

# load .env file
from dotenv import load_dotenv
from geo_model_evaluator import GeoModelEvaluator
from wandb_downloader import WandbDownloader

sys.path.insert(0, "../4_modeling")
from image_data_handler import ImageDataHandler

sys.path.insert(0, "../")
from data_loader import get_data_to_load, hash_filenames

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
WANDB_TOKEN = os.getenv("WANDB_TOKEN")
# Define where to run
env_path = "../../.env"
if not WANDB_TOKEN and os.path.exists(env_path):
    load_dotenv(env_path)
    WANDB_TOKEN = os.getenv("WANDB_TOKEN")

In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")

    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


# Set runs to update (should be of the same project, type and dataset (augmentation, size, ...))

In [5]:
entity = "nlp_ls"
project = "dspro2-predicting-region"
run_ids = ["zdriu52s", "upys7lpk", "nxtf1y1t", "upys7lpk", "ocaaj18u", "upys7lpk", "ocaaj18u", "zdriu52s", "nxtf1y1t", "34oughmz"]  # should all be of the same project, type and dataset (augmentation, size, ...)

wandb.login(key=WANDB_TOKEN) if WANDB_TOKEN else wandb.login()

# Get first run to get the config via the API
api = wandb.Api()
run = api.run(f"{entity}/{project}/{run_ids[0]}")
config = run.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: killusions. Use `wandb login --relogin` to force relogin


# Get config

In [6]:
NUMBER_OF_FILES = config["dataset_size"]
USE_MAPPED = config["mapped_data"]

image_size = config["input_image_size"]
predict_coordinates = config["predict_coordinates"]
predict_regions = config["predict_regions"]

# Decide on which gpu to run with best settings

In [7]:
BATCH_SIZE = 400

running_device = "colab_A100"

if running_device == "colab_T4":
    # Run unmapped images with low image resolution on colab
    BATCH_SIZE = 300

elif running_device == "colab_A100":
    # Run mapped images with high image resolution on colab
    BATCH_SIZE = 200

elif running_device == "gpuHub":
    # Run unmapped images with low image resolution on gpuHub
    BATCH_SIZE = 200

elif running_device == "gpuHub_augmentedv2":
    # Run unmapped images with low image resolution on gpuHub
    BATCH_SIZE = 100

# Loading data

In [8]:
# get list with local data and file paths
list_files, load_callback, additional_save_callback = get_data_to_load(loading_file="../3_data_preparation/04_data_cleaning/updated_data_list_more" if USE_MAPPED else "../3_data_preparation/04_data_cleaning/updated_data_list_non_mapped", file_location="../3_data_preparation/01_enriching/.data", image_file_location="../1_data_collection/.data", allow_new_file_creation=False, from_remote_only=True, download_link="default", limit=NUMBER_OF_FILES, shuffle_seed=42, allow_file_location_env=True, allow_json_file_location_env=True, allow_image_file_location_env=True, allow_download_link_env=True, return_load_and_additional_save_callback=True)

All local files: 705681
Relevant files: 705681
Limited files: 665572


In [9]:
NUMBER_OF_FILES = len(list_files) // 2
print(NUMBER_OF_FILES)

332786


## Processing and loading data

In [10]:
prediction_type = "regions" if predict_regions else ("coordinates" if predict_coordinates else "countries")

train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

data_augmentation = config["data_augmentation"]

preprocessing_config = {"data_augmentation": data_augmentation, "height": image_size[0], "width": image_size[1], "train_ratio": train_ratio, "val_ratio": val_ratio, "test_ratio": test_ratio}

augmented_transform = None  # Never used in this script
base_transform = transforms.Compose([transforms.Resize((image_size[0], image_size[1])), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [11]:
# Creating Dataloasders with the classes

# Hash the files list to get a unique identifier for the data
hashed_filenames = hash_filenames(list_files)

data_handler = ImageDataHandler(list_files, augmented_transform, base_transform, preprocessing_config, prediction_type, batch_size=BATCH_SIZE, train_ratio=train_ratio, val_ratio=val_ratio, test_ratio=test_ratio, cache=False, cache_load_callback=load_callback, cache_additional_save_callback=additional_save_callback, save_test_data=False, inspect_transformed=False, move_files=False, get_cache=True)
val_dataloader = data_handler.val_loader
country_to_index = data_handler.country_to_index
region_to_index = data_handler.region_to_index
region_index_to_middle_point = data_handler.region_index_to_middle_point
region_index_to_country_index = data_handler.region_index_to_country_index

# Load the country_to_index mapping and print the count of different countries
print("Dataset size:", NUMBER_OF_FILES)
print("Dataset identifier:", hashed_filenames)
print(f"Count of different countries: {len(country_to_index)}")
print(f"Count of different regions: {len(region_to_index)}")

In [ ]:
print("Number of val batches:", len(val_dataloader.dataset), "")

# Print first batch as an example, to see the structure
for images, coordinates, country_indices, region_indices in val_dataloader:
    print("Images batch shape:", images.shape)
    print("Coordinates batch shape:", coordinates.shape)
    print(coordinates[0])
    print("Country indices:", country_indices.shape)
    print(country_indices[0])
    print("Region handler:", region_indices.shape)
    print(region_indices[0])
    break

## Training

In [ ]:
model_type = config["model_name"]

if predict_coordinates:
    num_classes = 3
elif predict_regions:
    num_classes = len(region_to_index)
else:
    num_classes = len(country_to_index)

evaluator = GeoModelEvaluator(val_dataloader=val_dataloader, num_classes=num_classes, predict_coordinates=predict_coordinates, country_to_index=country_to_index, region_to_index=region_to_index, region_index_to_middle_point=region_index_to_middle_point, region_index_to_country_index=region_index_to_country_index, predict_regions=predict_regions if not predict_coordinates else None)

# For loading model
file_names_to_download = [".pth"]
# Countries from 81k more mapped dataset, keep in sync with evaluating_models.ipynb
countries_only = ["Albania", "Argentina", "Australia", "Austria", "Bangladesh", "Belgium", "Bolivia, Plurinational State of", "Botswana", "Brazil", "Bulgaria", "Cambodia", "Canada", "Chile", "Colombia", "Croatia", "Czechia", "Denmark", "Dominican Republic", "Ecuador", "Estonia", "Eswatini", "Finland", "France", "Germany", "Ghana", "Greece", "Guatemala", "Hungary", "India", "Indonesia", "Ireland", "Israel", "Italy", "Japan", "Kenya", "Korea, Republic of", "Kyrgyzstan", "Lao People's Democratic Republic", "Latvia", "Lesotho", "Lithuania", "Malaysia", "Malta", "Mexico", "Montenegro", "Netherlands", "New Zealand", "Nigeria", "North Macedonia", "Norway", "Peru", "Philippines", "Poland", "Portugal", "Romania", "Russian Federation", "Rwanda", "Senegal", "Serbia", "Singapore", "Slovakia", "Slovenia", "South Africa", "Spain", "Sri Lanka", "Sweden", "Switzerland", "Thailand", "T\u00fcrkiye", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay"]

for run_id in run_ids:
    api_run = api.run(f"{entity}/{project}/{run_id}")
    with wandb.init(id=run_id, project=project, entity=entity, resume=True) as run:
        # new line
        print()
        print(f"Evaluating for run {run.name} ({run.id})")
        # new line
        print()

        run_info = WandbDownloader.get_run_data(api=api, entity=entity, project=project, run=api_run, file_names=[".pth"])
        # Load the model
        evaluator.evaluate(model_type=model_type, model_path=run_info["files"]["best_model"], use_balanced_accuracy=True, second_balanced_on_countries_only=countries_only if NUMBER_OF_FILES > 100000 else None, accuracy_per_country=True, median_metric=True)